In [2]:
import winsound
from urllib3.connection import HTTPConnection
import socket
import numpy as np
from IPython.display import display, HTML
import opendota
import requests
import pandas as pd
import time
import tqdm.notebook as tqdm
import re
import plotly.express as px
import plotly.graph_objects as go
import ast
from tqdm.contrib.telegram import tqdm as tqdmt
import time
from plyer import notification
from datetime import timedelta

client = opendota.OpenDota() 


def pretty_print(df):
    pd.set_option("display.max_columns", None)
    pd.set_option("display.max_rows", None)
    pd.set_option("display.precision", 2)
    display(
        HTML(df.to_html(float_format="{:,}".format).replace("\\n", "<br>")))
    pd.reset_option("display.max_columns")
    pd.reset_option("display.max_rows")


HTTPConnection.default_socket_options = HTTPConnection.default_socket_options + [
    (socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1),
    (socket.SOL_TCP, socket.TCP_KEEPIDLE, 45),
    (socket.SOL_TCP, socket.TCP_KEEPINTVL, 10),
    (socket.SOL_TCP, socket.TCP_KEEPCNT, 6),
]
import datetime

last_week = datetime.datetime.today() - datetime.timedelta(days=9)

public_matches_H = client.explorer(
    f'select * from public_matches\
                                 where avg_rank_tier BETWEEN 9 AND 16 AND "lobby_type" = 7 AND start_time > {round(last_week.timestamp())} ORDER BY duration DESC LIMIT 5000'
)
public_matches_G = client.explorer(
    f'select * from public_matches\
                                 where avg_rank_tier BETWEEN 19 AND 26 AND "lobby_type" = 7 AND start_time > {round(last_week.timestamp())} ORDER BY duration DESC LIMIT 4999'
)
# df = pd.DataFrame(public_matches_H)
df = pd.concat(
    [pd.DataFrame(public_matches_H), pd.DataFrame(public_matches_G)], ignore_index=True
)
df_edit = df[:]


def get_match_data(data_type):
    df_match = pd.DataFrame(columns=data_type)
    print(f"currently getting data for = {data_type}")
    fail_num = 0
    for i in tqdm.tqdm(range(len(df_edit["match_id"]))):
        # try:
        list_a = []
        start = time.time()
        match_id = df_edit.loc[i, "match_id"]
        # data = requests.get(f"https://api.opendota.com/api/matches/{match_id}").json()
        data = client.get_match(match_id=match_id)
        for item in data_type:
            list_a.append(data[item])
        df_match.loc[len(df_match)] = list_a
        end = time.time()
        duration = end - start
        if duration < 1.05:
            time.sleep(1.05 - duration)
        # except:
        #     fail_num += 1
        #     print(f'fail number = {fail_num}, waiting 5 second')
        #     time.sleep(5)
        #     data = requests.get(f"https://api.opendota.com/api/matches/{match_id}").json()
        #     for item in data_type:
        #         list_a.append(data[item])
        #     df_match.loc[len(df_match)] = list_a

    return df_match

In [ ]:
import requests
import time
import concurrent.futures

img_urls = [
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1530224264768-7ff8c1789d79',
    'https://images.unsplash.com/photo-1564135624576-c5c88640f235',
    'https://images.unsplash.com/photo-1541698444083-023c97d3f4b6',
    'https://images.unsplash.com/photo-1522364723953-452d3431c267',
    'https://images.unsplash.com/photo-1513938709626-033611b8cc03',
    'https://images.unsplash.com/photo-1507143550189-fed454f93097',
    'https://images.unsplash.com/photo-1493976040374-85c8e12f0c0e',
    'https://images.unsplash.com/photo-1504198453319-5ce911bafcde',
    'https://images.unsplash.com/photo-1530122037265-a5f1f91d3b99',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1549692520-acc6669e2f0c'
]

t1 = time.perf_counter()


def download_image(img_url):
    img_bytes = requests.get(img_url).content
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        print(f'{img_name} was downloaded...')


with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(download_image, img_urls)


t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

In [20]:
df_edit

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster
0,7212306718,6063696596,False,1687522136,5700,NaN,NaN,7,22,15,2,272
1,7218080114,6068882731,True,1687869352,5385,NaN,NaN,7,22,13,3,182
2,7213444416,6064696286,True,1687590237,5209,NaN,NaN,7,22,13,3,151
3,7217003315,6067905504,True,1687793971,5036,NaN,NaN,7,22,14,7,227
4,7216179510,6067135769,True,1687742611,5008,NaN,NaN,7,22,13,2,251
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,7216245306,6067182653,False,1687749569,3375,NaN,NaN,7,22,25,2,151
9995,7215989411,6066956282,False,1687725392,3375,1282.0,1.0,7,22,25,3,241
9996,7215934105,6066907758,False,1687722306,3375,NaN,NaN,7,22,22,2,182
9997,7218752702,6069424506,False,1687899631,3375,NaN,NaN,7,22,22,2,251


In [25]:
def cluster1(cluster):
    new_cluster = cluster + 1
    return new_cluster
start = time.perf_counter()

df['new_cls'] = df.apply()
# for i in range(0,len(df_edit),3):
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         f1 = executor.submit(cluster1,df.loc[i,'cluster'])
#         f2 = executor.submit(cluster1,df.loc[i+1,'cluster'])
#         f3 = executor.submit(cluster1,df.loc[i+2,'cluster'])
        
#         df.loc[i,'new_cluster'] = f1.result()
#         df.loc[i+1,'new_cluster'] = f2.result()
#         df.loc[i+2,'new_cluster'] = f3.result()

finish = time.perf_counter()

print(finish-start)
display(df)

2.479698800000051


,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,new_cluster
0,7212306718,6063696596,False,1687522136,5700,NaN,NaN,7,22,15,2,272,273.0
1,7218080114,6068882731,True,1687869352,5385,NaN,NaN,7,22,13,3,182,183.0
2,7213444416,6064696286,True,1687590237,5209,NaN,NaN,7,22,13,3,151,152.0
3,7217003315,6067905504,True,1687793971,5036,NaN,NaN,7,22,14,7,227,228.0
4,7216179510,6067135769,True,1687742611,5008,NaN,NaN,7,22,13,2,251,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,7216245306,6067182653,False,1687749569,3375,NaN,NaN,7,22,25,2,151,152.0
9995,7215989411,6066956282,False,1687725392,3375,1282.0,1.0,7,22,25,3,241,242.0
9996,7215934105,6066907758,False,1687722306,3375,NaN,NaN,7,22,22,2,182,183.0
9997,7218752702,6069424506,False,1687899631,3375,NaN,NaN,7,22,22,2,251,252.0
